### Install packages

In [1]:
# Restart notebook after installing packages
#!pip install astropy numpy

## Load response functions for each detector unit 

In [2]:
import numpy
from astropy.io import fits

# RMF for each detector unit. 
with fits.open('ixpe_d1_obssim_v011.rmf') as hdu:
    matrix1=hdu[1].data['MATRIX']
    matrixt1=numpy.transpose(matrix1)
with fits.open('ixpe_d2_obssim_v011.rmf') as hdu:
    matrix2=hdu[1].data['MATRIX']
    matrixt2=numpy.transpose(matrix2)
with fits.open('ixpe_d3_obssim_v011.rmf') as hdu:
    matrix3=hdu[1].data['MATRIX']
    matrixt3=numpy.transpose(matrix3)
    
#MODF for each detector unit. 
with fits.open('ixpe_d1_obssim_mfact_v011.fits') as hdu:
    modf_du1=hdu[1].data['SPECRESP']
with fits.open('ixpe_d2_obssim_mfact_v011.fits') as hdu:
    modf_du2=hdu[1].data['SPECRESP']
with fits.open('ixpe_d3_obssim_mfact_v011.fits') as hdu:
    modf_du3=hdu[1].data['SPECRESP']
    
# ARF for each detector unit. 
with fits.open('ixpe_d1_obssim_v011.arf') as hdu:
    effae=(hdu[1].data['ENERG_LO']+hdu[1].data['ENERG_HI'])/2
    effa_du1=hdu[1].data['SPECRESP']
with fits.open('ixpe_d2_obssim_v011.arf') as hdu:
    effa_du2=hdu[1].data['SPECRESP']
with fits.open('ixpe_d3_obssim_v011.arf') as hdu:
    effa_du3=hdu[1].data['SPECRESP']

matrixtlist = [matrixt1, matrixt2, matrixt3]             
modflist    = [modf_du1, modf_du2, modf_du3]  
arflist     = [effa_du1, effa_du2, effa_du3]   

## Load simulated polarimetry data

In [3]:
folded_EVT_FILE_PATH = ['Caiazzo_Heyl_Figure_7C_pid3375_task_2_run_1_du1_Energy_folded.fits',
                        'Caiazzo_Heyl_Figure_7C_pid3375_task_2_run_1_du2_Energy_folded.fits',
                        'Caiazzo_Heyl_Figure_7C_pid3375_task_2_run_1_du3_Energy_folded.fits']

In [4]:
# events for each du 
with fits.open(folded_EVT_FILE_PATH[0]) as hdul_du1:
    qphoton_du1   = hdul_du1[1].data['Q']
    uphoton_du1   = hdul_du1[1].data['U']
    ephoton_du1   = hdul_du1[1].data['ENERGY']
    pphoton_du1   = hdul_du1[1].data['PHASE']
    phaphoton_du1 = hdul_du1[1].data['PHA'].astype(int)

with fits.open(folded_EVT_FILE_PATH[1]) as hdul_du2:
    qphoton_du2   = hdul_du2[1].data['Q']
    uphoton_du2   = hdul_du2[1].data['U']
    ephoton_du2   = hdul_du2[1].data['ENERGY']
    pphoton_du2   = hdul_du2[1].data['PHASE']
    phaphoton_du2 = hdul_du2[1].data['PHA'].astype(int)

with fits.open(folded_EVT_FILE_PATH[2]) as hdul_du3:
    qphoton_du3   = hdul_du3[1].data['Q']
    uphoton_du3   = hdul_du3[1].data['U']
    ephoton_du3   = hdul_du3[1].data['ENERGY']
    pphoton_du3   = hdul_du3[1].data['PHASE']
    phaphoton_du3 = hdul_du3[1].data['PHA'].astype(int)
    
# select 2-4 keV events in each du
indxdu1=(ephoton_du1<=4) & (ephoton_du1>=2)
edu1=ephoton_du1[indxdu1]
pdu1=pphoton_du1[indxdu1]
qdu1=qphoton_du1[indxdu1]
udu1=uphoton_du1[indxdu1]
phadu1=phaphoton_du1[indxdu1]

indxdu2=(ephoton_du2<=4) & (ephoton_du2>=2)
edu2=ephoton_du2[indxdu2]
pdu2=pphoton_du2[indxdu2]
qdu2=qphoton_du2[indxdu2]
udu2=uphoton_du2[indxdu2]
phadu2=phaphoton_du2[indxdu2]

indxdu3=(ephoton_du3<=4) & (ephoton_du3>=2)
edu3=ephoton_du3[indxdu3]
pdu3=pphoton_du3[indxdu3]
qdu3=qphoton_du3[indxdu3]
udu3=uphoton_du3[indxdu3]
phadu3=phaphoton_du3[indxdu3]

# prepare data list to perform analysis 
qdulist     = [qdu1,qdu2,qdu3]
udulist     = [udu1,udu2,udu3]
edulist     = [edu1,edu2,edu3]
pdulist     = [pdu1,pdu2,pdu3]
phadulist   = [phadu1,phadu2,phadu3]   
pha_list=numpy.arange(matrixt1.shape[0])

## Rotation Vector Model and likelihood function 

For simplicity, we stock  all photon data and minimize the likehood function assuming equal response modf for all three detector units

In [5]:
qok = numpy.concatenate((qdulist[0],qdulist[1],qdulist[2]))
uok = numpy.concatenate((udulist[0],udulist[1],udulist[2]))
pok = numpy.concatenate((pdulist[0],pdulist[1],pdulist[2]))
eok = numpy.concatenate((edulist[0],edulist[1],edulist[2]))
modfok = numpy.interp(eok, effae, modf_du1)

In [6]:
from scipy.optimize import minimize

def angfunk(alpha,beta,phase):
    tanhalfC=numpy.tan(phase*numpy.pi)
    halfamb=numpy.radians(alpha-beta)/2
    halfapb=numpy.radians(alpha+beta)/2
    halfAmB=numpy.arctan(numpy.sin(halfamb)/numpy.sin(halfapb)*tanhalfC)
    halfApB=numpy.arctan(numpy.cos(halfamb)/numpy.cos(halfapb)*tanhalfC)
    return -(halfApB-halfAmB)
    
def likelihood_rvm(alpha,beta,pos_ang,phase0,degm):
    ang=angfunk(alpha,beta,pok-phase0)+numpy.radians(pos_ang)
    qm=numpy.cos(2*ang)
    um=numpy.sin(2*ang)
    s=numpy.sum(numpy.log(1+0.5*modfok*degm*(qok*qm+uok*um)))
    return s

RVMres=minimize(lambda x: -likelihood_rvm(x[0],x[1],x[2],x[3],numpy.exp(-x[4]*x[4])), x0=[45,45,30,0.5,0.5],method='Nelder-Mead')
print('Solution Rotating Vector Model:\n')
print(' alpha =  %8.3f\n beta = %10.3f\n pos_ang = %6.3f\n phase0 = %7.3f\n degm = %9.3f'  % (RVMres.x[0], RVMres.x[1], RVMres.x[2], RVMres.x[3], RVMres.x[4]))

Solution Rotating Vector Model:

 alpha =    51.209
 beta =     43.005
 pos_ang =  0.042
 phase0 =   0.500
 degm =     0.495


## Unbinned analysis using spectro-polarimetric model of Caiazzo
Load model and for simplicity we calculate predicted number of event (npred) asuming equal RMF for all three dectectors units 

In [10]:
import sys
from ixpe_file_class_v2 import ixpe_file_class
from scipy.integrate import simps
from spline import xInterpolatedBivariateSpline #from ixpeobssim (Baldini+2022)

model=ixpe_file_class('Polarization_wQED_7k_2c.txt',alpha=50,beta=42,
                      normflux='HerX1_NuSTAR.txt',intensity_energy_units=False,enerlist=effae)

SIM_DURATION = 100000.
cnts=numpy.interp(effae,model.enerlist,numpy.mean(model.flux,axis=-1))*(effa_du1+effa_du2+effa_du3)
ratepred=simps(cnts,effae)
npred=SIM_DURATION*ratepred
cnt_conv=numpy.dot(matrixt1,cnts)

False
False


#### Unbinned analysis using spectro-polarimetric model, accounting for response functions of all three detectors: energy dispersion, effective area, and modulation factor 

In [8]:

def likelihoodspin(alpha,beta,angshift,deltadeg):
    #print('%0.5f %0.5f %0.5f %0.5f' % (alpha,beta,angshift,deltadeg))
    model.reset_geometry(alpha,beta)
    flux_conv=numpy.dot(matrixt1,model.flux)
    fdulist = []
    for matrixtn, arfdu, funmodf, qdu, udu, edu, pdu, phadu in zip(matrixtlist,  arflist, modflist, qdulist, udulist, edulist, pdulist, phadulist):
        flux_conv=numpy.dot(matrixtn,numpy.transpose(arfdu*numpy.transpose(model.flux)))
        u_data=numpy.dot(matrixtn, numpy.transpose((arfdu*funmodf)*numpy.transpose(model.flux*model.u_data*model.pol_deg_data)))
        q_data=numpy.dot(matrixtn, numpy.transpose((arfdu*funmodf)*numpy.transpose(model.flux*model.q_data*model.pol_deg_data)))
        u_data/=flux_conv
        q_data/=flux_conv
        u_spline = xInterpolatedBivariateSpline(pha_list, model.phase, u_data,kx=3, ky=3)
        q_spline = xInterpolatedBivariateSpline(pha_list, model.phase, q_data,kx=3, ky=3)
        um=u_spline(phadu,pdu)
        qm=q_spline(phadu,pdu)

        pcos2d = (qdu*qm + udu*um)
        psin2d = (qdu*um - udu*qm)       
        pcos2tot = pcos2d*numpy.cos(2*numpy.radians(angshift)) - psin2d*numpy.sin(2*numpy.radians(angshift))

        fdu= 1 + 0.5*(1.+deltadeg)*pcos2tot

        fdulist=numpy.concatenate((fdulist,fdu))

    return numpy.sum(numpy.log(fdulist))-npred

#### Minimization of likelihood function including energy dispersion may take a few minutes

In [9]:
from scipy.optimize import minimize

MDLres=minimize((lambda x: -likelihoodspin(x[0],x[1],x[2],x[3])), x0=[51,41,0.1,0.1],method='Nelder-Mead')
print('Solution Spectro-polarimetric Model:\n')
print(' alpha =  %8.3f\n beta = %10.3f\n angshift = %.3f\n deltadeg = %.3f'  % (MDLres.x[0], MDLres.x[1], MDLres.x[2], MDLres.x[3]))

Solution Spectro-polarimetric Model:

 alpha =    51.087
 beta =     42.901
 angshift = 0.050
 deltadeg = 0.123
